In [1]:
#Python Libraries for Mongos Database
import pymongo
from pymongo import MongoClient

#Serializing to a file
import _pickle as pickle

In [2]:
#Python Library for Dataframe usage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Python Libraries for Graph
import networkx as nx

In [3]:
#accessing mongoDB
client = MongoClient()
database = client['food_map']   # Database name (to connect to)
collections = database['flavor_molecules']

#Getting the dataset from MongoDB into Pandas
flavorDB_pandas = pd.DataFrame(list(collections.find()))

In [4]:
#accessing mongoDB
client = MongoClient()
database = client['food_map']   # Database name (to connect to)
collections = database['flavor_molecules']

#Getting the dataset from MongoDB into Pandas
flavorDB_pandas = pd.DataFrame(list(collections.find()))
flavorDB_pandas.rename(index=str, columns={"catgeroy": "category"})
#Making each list into a set
flavorDB_pandas["set_molecules"] = flavorDB_pandas["molecules"].apply(lambda row: set(row))
flavorDB_pandas["set_molecules_ID"] = flavorDB_pandas["molecule_IDs"].apply(lambda row: set(row))
flavorDB_pandas["molecule_quantity"] = flavorDB_pandas["molecule_IDs"].apply(lambda row: len(row))
flavorDB_pandas["ingredient"] = flavorDB_pandas["ingredient"].apply(lambda row: row.lower())

In [5]:
#Importing a set that has all the ingredients with recipes
#Opening the pickled file
pickle_in = open("./data/ingredients_with_recipes.pickle","rb")
#Getting the dictionary from the pickle
set_of_ing_with_recipe = pickle.load(pickle_in)

FileNotFoundError: [Errno 2] No such file or directory: './data/ingredients_with_recipes.pickle'

In [ ]:
flavorDB_pandas= flavorDB_pandas[flavorDB_pandas["ingredient"].isin(set_of_ing_with_recipe)]

In [ ]:
flavorDB_pandas.head()

In [ ]:
with open('flavorDB_pandas.pickle', 'wb') as file:
    file.write(pickle.dumps(flavorDB_pandas))
    file.close()

In [ ]:
print("total number of ingredients: ", len(flavorDB_pandas["ingredient"]))

In [ ]:
flavorDB_pandas["catgeory"].unique()

In [ ]:
print("the mean number of flavor molecules per ingredient: ", flavorDB_pandas["molecule_quantity"].mean())

In [ ]:
print("the median number of flavor molecules per ingredient: ", flavorDB_pandas["molecule_quantity"].median())

In [ ]:
plt.hist(flavorDB_pandas["molecule_quantity"])

In [ ]:
full_G=nx.Graph()

#iterate through each row of flavorDB based on if index is in random sample
for index, row in flavorDB_pandas.iterrows():
    #set of the ingredient from the "rows"
    set1= row["set_molecules"]
    #name of the ingredient from the "rows" 
    ingredient_1 = row["ingredient"]

    if True: # or len(set1) < 5: # To set if you want to consider all ingredients
        for molecule in set1:
            full_G.add_node(ingredient_1)
            full_G.node[ingredient_1]["ingredient_node"] = True
            full_G.add_node(molecule)
            full_G.node[molecule]["molecule_node"] = True
            full_G.add_edge(ingredient_1, molecule)

In [ ]:
ingredient_nodes = nx.get_node_attributes(full_G, 'ingredient_node').keys()
molecule_nodes = nx.get_node_attributes(full_G, 'molecule_node').keys()

print("total number of nodes: ", full_G.number_of_nodes())
print("total number of ingredient nodes: ", len(ingredient_nodes))
print("total number of molecule nodes: ", len(molecule_nodes))

In [ ]:
print("total number of edges: ", full_G.number_of_edges())
print("total possible number of edges: 1779 molecules * 935 ingredients = 1663365")

## Demonstration Using Milk and Coffee

In [ ]:
demo_G=nx.Graph()

#iterate through each row of flavorDB based on if index is in random sample
for index, row in flavorDB_pandas.iterrows():
    #set of the ingredient from the "rows"
    set1= row["set_molecules"]
    #name of the ingredient from the "rows" 
    ingredient_1 = row["ingredient"]

    if ingredient_1 in ["coffee", "milk"]:
        for molecule in set1:
            # flavor_matrix_df[ingredient_1][molecule] = {'weight': 1}
            demo_G.add_node(ingredient_1)
            demo_G.node[ingredient_1]["ingredient_node"] = True
            demo_G.add_node(molecule)
            demo_G.node[molecule]["molecule_node"] = True
            demo_G.add_edge(ingredient_1, molecule)

In [ ]:
ingredient_nodes = nx.get_node_attributes(demo_G, 'ingredient_node').keys()
molecule_nodes = nx.get_node_attributes(demo_G, 'molecule_node').keys()

In [ ]:
pos=nx.spring_layout(demo_G)
nx.draw_networkx_nodes(demo_G,pos,
                       nodelist=ingredient_nodes,
                       node_color='r',
                       node_size=500,
                   alpha=0.8)
nx.draw_networkx_nodes(demo_G,pos,
                       nodelist=molecule_nodes,
                       node_color='b',
                       node_size=500,
                   alpha=0.8)

In [ ]:
#Plotting the Graph 
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
nx.draw_networkx(demo_G, ax=ax)
plt.show()

In [ ]:
centrality = nx.eigenvector_centrality(demo_G)

In [ ]:
centrality

## Another Demo

In [ ]:
demo_G=nx.Graph()

#iterate through each row of flavorDB based on if index is in random sample
for index, row in flavorDB_pandas.iterrows():
    #set of the ingredient from the "rows"
    set1= row["set_molecules"]
    #name of the ingredient from the "rows" 
    ingredient_1 = row["ingredient"]

    if ingredient_1 in ["lobster", "beef"]:
        for molecule in set1:
            # flavor_matrix_df[ingredient_1][molecule] = {'weight': 1}
            demo_G.add_node(ingredient_1)
            demo_G.node[ingredient_1]["ingredient_node"] = True
            demo_G.add_node(molecule)
            demo_G.node[molecule]["molecule_node"] = True
            demo_G.add_edge(ingredient_1, molecule)

In [ ]:
ingredient_nodes = nx.get_node_attributes(demo_G, 'ingredient_node').keys()
molecule_nodes = nx.get_node_attributes(demo_G, 'molecule_node').keys()

In [ ]:
node_to_edges_dict = {}
for node in demo_G.node:
    node_to_edges_dict[node] = len(demo_G.edges(node))
node_to_edges_dict

In [ ]:
connected_nodes_only = [k for k,v in node_to_edges_dict.items() if v >= 2]


In [ ]:
connected_edges_only = []
for node1 in connected_nodes_only:
    for node2 in connected_nodes_only :
        if node1 != node2:
            edge = tuple([node1, node2])
            connected_edges_only.append(edge)

In [ ]:
pos=nx.spring_layout(demo_G)
nx.draw_networkx_nodes(demo_G,pos,
                       nodelist=ingredient_nodes,
                       node_color='r',
                       node_size=500,
                   alpha=0.8)
nx.draw_networkx_nodes(demo_G,pos,
                       nodelist=molecule_nodes,
                       node_color='b',
                       node_size=500,
                   alpha=0.8)

In [ ]:
connected_nodes_only

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
nx.draw_networkx(demo_G, edgelist = connected_edges_only, nodelist=connected_nodes_only, ax=ax)
plt.show()